In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

#cd command needs '%' not '!', while other Unix commands need '!'
%cd ./drive/My\ Drive/summer\ vacation 3
!pwd
!ls -l

Mounted at /content/drive
/content/drive/My Drive/summer vacation 3
/content/drive/My Drive/summer vacation 3
total 1825
drwx------ 2 root root    4096 Aug  3 09:25  data
-rw------- 1 root root   17852 Aug  9 09:58  LOL_data_model.ipynb
-rw------- 1 root root   29690 Aug 12 11:02 'mushroom CNN.ipynb'
-rw------- 1 root root   25285 Aug 15 09:45 'pytorch 버섯 CNN분류.ipynb'
-rw------- 1 root root 1790625 Aug 13 08:30 '버섯데이터 전이학습.ipynb'


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
import random
import torch.optim as optim
import tensorflow as tf
import tensorflow_hub as hub
import warnings
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
from transformers import AutoFeatureExtractor, SwinForImageClassification
from torchvision import models
warnings.filterwarnings('ignore')

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
image_paths = []

for relative_path1 in os.listdir('/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms'):
    for relative_path2 in tqdm(os.listdir(os.path.join('/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms', relative_path1))):
#         print(relative_path2)
        full_path = os.path.join('/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms', relative_path1, relative_path2)
        image_paths.append(full_path)

print(len(image_paths))

100%|██████████| 353/353 [00:00<00:00, 156097.98it/s]

6714


In [6]:
image_paths[:5]

['/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/079_6ZA8RSmhL-k.jpg',
 '/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/080_Ye1LzJ2aCoI.jpg',
 '/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/016_S-Z6ZIo2G3k.jpg',
 '/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/046_55Y02Q5s8MQ.jpg',
 '/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/100_w6tcyq1AMVk.jpg']

In [7]:
test_image_paths = random.sample(image_paths, k=int(len(image_paths) * 0.15))
train_image_paths = [x for x in image_paths if x not in test_image_paths]
print(len(test_image_paths), len(train_image_paths))

1007 5707


In [8]:
class_name = sorted(os.listdir('/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms'))
print(class_name)
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_name)}
print(class_to_idx)

['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
{'Agaricus': 0, 'Amanita': 1, 'Boletus': 2, 'Cortinarius': 3, 'Entoloma': 4, 'Hygrocybe': 5, 'Lactarius': 6, 'Russula': 7, 'Suillus': 8}


In [9]:
class MushroomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __getitem__(self, idx):
        try:
            label = class_to_idx[self.image_paths[idx].split('/')[-2]]
            image = Image.open(self.image_paths[idx]).convert("RGB")
#             image = np.clip(image, 0, 1)
#             image = Image.fromarray((image * 255).astype(np.uint8))
            if self.transform is not None:
                image = self.transform(image)
            return image, label
        except:
            image = Image.open('/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Agaricus/000_ePQknW8cTp8.jpg').convert("RGB")
            label = 0
            if self.transform is not None:
                image = self.transform(image)
            return image, label

    def __len__(self):
        return len(self.image_paths)

In [11]:
train_data = MushroomDataset(train_image_paths, transform=train_transform)
test_data = MushroomDataset(test_image_paths, transform=test_transform)

print(len(train_data), len(test_data))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

5707 1007


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [13]:
label_dict = {}
for path in train_image_paths:
    label_name = path.split('/')[-2]
    label_dict[class_to_idx[label_name]] = label_dict.get(class_to_idx[label_name], 0) + 1
print(sorted(label_dict.items()))

[(0, 296), (1, 632), (2, 932), (3, 698), (4, 314), (5, 276), (6, 1323), (7, 975), (8, 261)]


In [14]:
num = []
weights = []
final_weights = []
for i in range(9):
    num.append(label_dict[i])
for i in range(9):
    weights.append(1 / num[i] * sum(num))
for i in range(9):
    final_weights.append(weights[i] / max(weights))
print(final_weights)

[0.8817567567567569, 0.4129746835443039, 0.2800429184549357, 0.3739255014326648, 0.8312101910828027, 0.9456521739130436, 0.19727891156462588, 0.26769230769230773, 1.0]


In [15]:
num_classes = 9
criterion = nn.CrossEntropyLoss().to(device)

분류

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 경로
training_dir = "/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms"

# 전체 이미지를 1/255로 스케일 조정
train_datagen = ImageDataGenerator(rescale=1/255)

# 훈련 이미지를 244*244 크기로 다시 조정하고, 레이블을 부여한 배치 데이터 생성
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(244, 244),
    # 버섯을 구분하는 다중 클래스 분류이므로 categorical 사용
    class_mode='categorical'
)

# 레이블 확인
print(train_generator.class_indices)

Found 6714 images belonging to 9 classes.
{'Agaricus': 0, 'Amanita': 1, 'Boletus': 2, 'Cortinarius': 3, 'Entoloma': 4, 'Hygrocybe': 5, 'Lactarius': 6, 'Russula': 7, 'Suillus': 8}


In [17]:
model = tf.keras.models.Sequential([
    # 입력값은 244*244 크기의 RGB 이미지
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(244, 244, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    # 분류 대상이 9개이므로 아홉개의 뉴런을 사용
    tf.keras.layers.Dense(9, activation='sigmoid')
])

In [18]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [19]:
# 훈련 시작
model.fit(train_generator, epochs=10)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image

테스트 예시

In [ ]:
# 테스트 이미지 가져오기
sample_images = [
    ['/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Agaricus/000_ePQknW8cTp8.jpg'],
    ['/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Amanita/000_ePQknW8cTp8.jpg'],
    ['/content/drive/MyDrive/summer vacation 3/data/archive/Mushrooms/Boletus/000_ePQknW8cTp8.jpg']
    ]

In [ ]:
for fn in sample_images:
    # matplotlib을 이용하여 이미지 출력
    plt.imshow(mpimg.imread(fn))
    plt.show()

    # Keras에 이미지를 300*300 크기로 불러오기
    img = tf.keras.utils.load_img(fn, target_size=(300, 300))
    # 이미지를 2D 배열로 변환
    x = tf.keras.utils.img_to_array(img)
    print("2D 배열 shape : ", x.shape)
    # 모델의 input_shape가 (300, 300, 3)이므로 이 모양으로 변환
    x = np.expand_dims(x, axis=0)
    print("3D 배열 shape : ", x.shape)

    classes = model.predict(x)

    print("모델 출력 : ", classes[0][0])
    if (classes[0][0] ==0):
        print(fn + "는 Agaricus입니다.")
    elif (classes[0][0] ==1):
        print(fn + "는 Amanita입니다.")
    elif (classes[0][0] ==2):
        print(fn + "는 Boletus입니다.")
    elif (classes[0][0] ==3):
        print(fn + "는 Cortinarius입니다.")
    elif (classes[0][0] ==4):
        print(fn + "는 Entoloma입니다.")
    elif (classes[0][0] ==5):
        print(fn + "는 Hygrocybe입니다.")
    elif (classes[0][0] ==6):
        print(fn + "는 Lactarius입니다.")
    elif (classes[0][0] ==7):
        print(fn + "는 Russula입니다.")
    else:
        print(fn + "는 Suillus입니다.")